Шаблон для скрейперов

## Библиотеки и настройки

In [1]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# для работы с буфером обмена
import pyperclip

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from random import randint
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# путь к файлам

file_path = 'C:/00_Data/temp_data/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [5]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

Рабочих прокси — 9


In [6]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

Номер прокси, от 0 до 8
5


In [7]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [8]:
# пока не придумал нормального способа авторизации

pyperclip.copy(proxy_list[proxy_number][1])

print('Login: в буфере обмена')
print('Pass: ' + proxy_list[proxy_number][2])

Login: в буфере обмена
Pass: EJ1SMvdd


In [9]:
# запускаю драйвер, проверяю настройки

#s=Service('C:/Users/bgi/Anaconda3/chromedriver/chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки на объявления

In [10]:
forum = input()

149


In [11]:

start_url = f'https://forum.ww2.ru/index.php?showforum={forum}'

arc_url = '&topicfilter=locked'

In [12]:
full_url = start_url

In [13]:
full_url = start_url + arc_url

In [14]:
driver.get(full_url)
sleep(3)

In [15]:
adv_number = driver.find_element(By.CLASS_NAME, 'ipsType_pagetitle').find_element(By.CLASS_NAME, 'desc').text

In [16]:
category = driver.find_element(By.CLASS_NAME, 'ipsType_pagetitle').text.replace(' ' + adv_number, '')

In [17]:
page_number = math.ceil(int(adv_number) / 60)

print('Количество объявлений: ' + adv_number)
print('Количество страниц: ' + str(page_number))

Количество объявлений: 1760
Количество страниц: 30


In [ ]:
data_url = [] # список для данных

for page in tqdm(range(1, page_number + 1)):
    
    driver.get(full_url + '&page=' + str(page))
    sleep(2)

    topic_blocks = driver.find_elements(By.TAG_NAME, 'tr[itemscope]')

    for topic in topic_blocks:
        name = topic.find_element(By.TAG_NAME, 'span[itemprop="name"]').text
        topic_url = topic.find_element(By.TAG_NAME, 'a[itemprop="url"]').get_attribute('href')
        start_date = topic.find_element(By.TAG_NAME, 'span[itemprop="dateCreated"]').text

        try:
            price = topic.find_element(By.CLASS_NAME, 'ebay_desc').text.replace(' ₽', '')

        except:
            price = 'none'
            
        try:
            views = (re.sub(r'.+\n', '', topic.find_element(By.CLASS_NAME, 'col_f_views')
                            .text).replace(' Просмотров', '').replace(' ', ''))
            
        except:
            views = 'none'
            

        try:
            price_type = topic.find_element(By.CLASS_NAME, 'ipsBadge_direct').text.lower()

        except:
            try:
                price_type = topic.find_element(By.CLASS_NAME, 'ipsBadge_auction').text.lower()

            except:
                price_type = 'none'

        update_date = 'none'
        item_number = 'none'

        for item_span in topic.find_elements(By.CLASS_NAME, 'desc'):
            if 'поднята' in item_span.text:
                update_date = re.sub(r'.+поднята ', '', item_span.text)

            if 'наличии' in item_span.text:
                item_number = item_span.text.replace('В наличии: ', '')

        data_url.append([category, name, price, price_type, item_number, start_date, update_date, views, topic_url])

In [ ]:
# преобразовываю в датафрейм

head = ['category', 'name', 'price', 'price_type', 'item_number',
        'start_date', 'update_date', 'views', 'topic_url']

work_url = pd.DataFrame(data_url, columns=head)

In [ ]:
work_url.shape

In [20]:
work_url.sample()

,category,name,price,price_type,item_number,start_date,update_date,views,topic_url
11270,Архив раздела Германия,баробан для сигнального нагана,none,none,none,14 май 2014,none,119,https://forum.ww2.ru/index.php?showtopic=4166202


In [18]:
work_url = pd.read_excel(file_path + 'work_url_aks_MMG_ge_a.xlsx')

In [19]:
category = work_url.category[0]
category

'Архив раздела Германия'

## Собираю описание и данные со страницы объявления

In [21]:
data = [] # список для данных

In [30]:
for url in tqdm(work_url.topic_url[8900:9001]):
    
    driver.get(url)
    
    sleep(randint(1, 3))
    
    if driver.current_url != start_url:
        
        try:
            if 'Продавец считает лот подлинным' in driver.find_element(By.CLASS_NAME, 'ipsList_data').text:
                authenticity = 1
            
            else:
                authenticity = 0
                
        except:
            pass
            
        # описание    
        try:
            description = driver.find_element(By.ID, 'ww2itemDesc').text
        
        except:
            description = 'none'

        # информация о продавце
        try:
            user_details = driver.find_element(By.CLASS_NAME, 'user_details').text

        except:
            user_details = 'none'

        try:
            nickname = driver.find_element(By.ID, 'username_e').text

        except:
            nickname = 'none'

        try:
            seller_status = driver.find_element(By.CLASS_NAME, 'ipsBadge').text

        except:
            seller_status = 'none'

        seller_group = driver.find_element(By.CLASS_NAME, 'group_title').text

        try:
            rate_pos = driver.find_element(By.CLASS_NAME, 'main_green').text

        except:
            rate_pos = 'none'

        try:
            rate_neg = driver.find_element(By.CLASS_NAME, 'main_red').text

        except:
            rate_neg = 'none'

        try:
            watching = int(
                re.findall(r'Наблюдают: \d+', driver.find_element(By.CLASS_NAME, 'ipsForm_vertical').text)[0]
                .replace('Наблюдают: ', ''))

        except:
            watching = 'none'


        try:
            temp_dict = {}
            vertical_blocks = (driver.find_element(By.CLASS_NAME, 'ipsForm_vertical')
                               .find_elements(By.CLASS_NAME, 'ipsField_primary'))

            for item in vertical_blocks:


                try:
                    title = item.find_element(By.TAG_NAME, 'h4').text
                    value = item.find_element(By.TAG_NAME, 'span').text

                    temp_dict[title] = value

                except:
                    pass

            country = temp_dict['Страна продавца']
            region = temp_dict['Расположение лота']

            try:
                delivery = temp_dict['Условия доставки']

            except:
                delivery = 'none'

            try:
                delivery_cost = temp_dict['Cтоимость доставки']

            except:
                delivery_cost = 'none'

            payment = temp_dict['Условия оплаты']

        except:
            pass
    
        data.append([url,
                 description,
                 authenticity, watching,
                 country, region,
                 delivery, delivery_cost, payment,
                 nickname, seller_status, seller_group,
                 rate_pos, rate_neg,
                 temp_dict, user_details])
    
    else:
        print('заблокирован')

  0%|          | 0/101 [00:00<?, ?it/s]

In [38]:
# преобразовываю в датафрейм

head = ['topic_url',
        'description',
        'authenticity', 'watching',
        'country', 'region',
        'delivery', 'delivery_cost', 'payment',
        'nickname', 'seller_status', 'seller_group',
        'rate_pos', 'rate_neg',
        'temp_dict', 'user_details']

work_data = pd.DataFrame(data, columns=head)

In [39]:
work_data.shape

(2095, 16)

In [40]:
work_data.head(3)

,topic_url,description,authenticity,watching,country,region,delivery,delivery_cost,payment,nickname,seller_status,seller_group,rate_pos,rate_neg,temp_dict,user_details
0,https://forum.ww2.ru/index.php?showtopic=4724027,...........,1,0,Россия,Новгородская обл. Парфино,Почта России,300 ₽,Оплата на карту,*******,none,Пользователи,90,-0,"{'Страна продавца': 'Россия', 'Расположение ло...",*******\nПользователи\n1 570 сообщений\nНа фор...
1,https://forum.ww2.ru/index.php?showtopic=4721463,Продам запчасти к М-24. Стоимость - 5500 руб. ...,1,0,Россия,Петрозаводск,Транспортная компания,none,Оплата на карту,*******,none,Пользователи,0,-0,"{'Страна продавца': 'Россия', 'Расположение ло...",*******\nПользователи\n114 сообщений\nНа форум...
2,https://forum.ww2.ru/index.php?showtopic=4719970,Продам ящик МГ ранний с остатками камо\nСостоя...,1,1,Россия,Тверская обл,Почта России,none,Оплата на карту,*******,none,Пользователи,22,-0,"{'Страна продавца': 'Россия', 'Расположение ло...",*******\nПользователи\n3 078 сообщений\nНа фор...


In [41]:
work = work_url.merge(work_data, how='left', on='topic_url')

In [42]:
work.sample()

,category,name,price,price_type,item_number,start_date,update_date,views,topic_url,description,...,delivery,delivery_cost,payment,nickname,seller_status,seller_group,rate_pos,rate_neg,temp_dict,user_details
15041,Архив раздела Германия,часть mp38/40 1941 г,none,none,none,08 май 2016,none,893,https://forum.ww2.ru/index.php?showtopic=4631778,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
work = work[['topic_url', 'category', 'name', 'description',
             'price', 'price_type', 'item_number',
             'country', 'region',
             'start_date', 'update_date', 'views', 'authenticity',
             'watching', 'delivery', 'delivery_cost', 'payment',
             'nickname', 'seller_status', 'seller_group',
             'rate_pos', 'rate_neg', 'user_details',
             'temp_dict']]

In [44]:
# Записываю в файл

work.to_excel(file_path + 'ww2_' + category + '7.xlsx', encoding='utf-8', index=False)

In [45]:
driver.quit()